In [49]:
#import all the required libraries
import pandas as pd
import selenium
from selenium import webdriver
from bs4 import BeautifulSoup
import requests
import time
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains

- Q1. Write a python program which searches all the product under a particular product from www.amazon.in. The product to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search for guitars.

In [44]:
driver=webdriver.Chrome("chromedriver.exe") 

In [45]:
url = "https://www.amazon.in/ref=nav_logo"
driver.get(url)

In [46]:
requests.get('https://www.amazon.in/ref=nav_logo')

<Response [200]>

In [47]:
# entering “headphones” in “Search” field 
search_field_search=driver.find_element_by_id("twotabsearchtextbox") #headphones search bar
search_field_search.send_keys("headphones")


# clicking the search button
search_button=driver.find_element_by_id("nav-search-submit-button")
search_button.click()

- Question- 2 - In the above question, now scrape the following details of each product listed in first 3 pages of your search results and save it in a dataframe and csv. In case if any product has less than 3 pages in search results

- then scrape all the products available under that product vertical. 

- Details to be scraped are: "Brand Name", "Name of the Product", "Rating", "No. of Ratings", "Price", "Return/Exchange", "Expected Delivery", "Availability", "Other Details" and “Product URL”. 

- In case, if any of the details are missing for any of the product then replace it by “-“.

In [48]:
next_button=[]
for i in driver.find_elements_by_xpath("//ul[@class='a-pagination']/li/a"): #scraping the list of buttons from the page
    next_button.append(i.get_attribute("href"))

In [51]:
#creating the empty list
brand=[]
product=[]
price=[]
rating_number=[]
product_url=[]
ratings=[]
return_exchange=[]
available=[]
delivery=[]
other=[]

    
urls=[]
for i in next_button[:3]:
    driver.get(i)
    
    purl=driver.find_elements_by_xpath("//a[@class='a-link-normal a-text-normal']")   
    for i in purl:
        product_url.append(i.get_attribute('href'))
    
    #Finding Product Urls
    prod_url = driver.find_elements_by_xpath("//a[@class='a-link-normal a-text-normal']")
    for i in prod_url:
        urls.append(i.get_attribute('href'))
    
        
    nxt_button=driver.find_element_by_class_name('a-last')
    nxt_button.click()

time.sleep(2)   
for i in urls:
    driver.get(i)
    
     #Finding brand_name
    try:       
        brand_name=driver.find_element_by_xpath("//*[@id='productOverview_feature_div']/div/table/tbody/tr[3]/td[2]/span")#locating the product
        brand.append(brand_name.text)#appending the ratings in Ratings list   
    except NoSuchElementException:
        brand.append("--")#appending the -- if no product is there 
        
    #Finding product
    try:       
        product_name=driver.find_element_by_xpath("//span[@id='productTitle']")#locating the product
        product.append(product_name.text)#appending the product name in Product list   
    except NoSuchElementException:
        product.append("--")#appending the -- if no product is there 
        
    try:       
        Price=driver.find_element_by_xpath("//td[@class='a-span12']")#locating the product
        price.append(Price.text)#appending the ratings in Ratings list   
    except NoSuchElementException:
        price.append("--")#appending the -- if no product is there 
         
    
    #Finding ratings_number
    try:       
        rating_n=driver.find_element_by_xpath("//span[@id='acrCustomerReviewText']") #locating the rating
        rating_number.append(rating_n.text)#appending the rating_number in Ratings list   
    except NoSuchElementException:
        rating_number.append("--") #appending the -- if no rating number is there
    
    #Rating
    try:       
        rating=driver.find_element_by_xpath("//span[@class='a-size-base a-nowrap']/span")#locating the rating
        ratings.append(rating.text) #appending the ratings in Ratings list   
    except NoSuchElementException:
        ratings.append("--")  #appending the -- if no rating is there 
           
    #Finding Day of delivery
    try:       
        Del=driver.find_element_by_xpath("//*[@id='ddmDeliveryMessage']/b")   #locating the delivery
        delivery.append(Del.text)#appending the delivery in list    
    except NoSuchElementException :
        delivery.append("--")#appending the -- if no date is there
        
    #Finding product decription    
    try:       
        other_info=driver.find_element_by_xpath("//ul[@class='a-unordered-list a-vertical a-spacing-mini']")#locating the info
        other.append(other_info.text)#appending the other information in list    
    except NoSuchElementException :
        other.append("--")#appending the -- if no information is there  
    
     #Finding Availability     
    try:       
        ava=driver.find_element_by_xpath("//span[@class='a-size-medium a-color-success']")#locating the Availability
        available.append(ava.text)#appending the other Availability in list    
    except NoSuchElementException :
        available.append("--")#appending the -- if no Availability is there  
    
    # Return or exchange  
    try:       
        retrn=driver.find_element_by_xpath("//*[@id='RETURNS_POLICY']/span/div[2]/a")#locating the return or exchange
        return_exchange.append(retrn.text)#appending the return_exchange in list
    except :
        return_exchange.append("--")#appending the -- if no  return_exchange is there
time.sleep(2)        
time.sleep(3)
        

In [60]:
#creating a dataframe
df=pd.DataFrame()
df['Brand']=brand
df['Name of the Product']=product
df['Rating']=ratings
df['No. of Ratings']=rating_number
df['Price']=price
df['Return/Exchange']=return_exchange
df['Expecte Delivery']=delivery
df['Availability']=available
df['Other Details']=other
df['Product url']=product_url

In [61]:
df.head()

,Brand,Name of the Product,Rating,No. of Ratings,Price,Return/Exchange,Expecte Delivery,Availability,Other Details,Product url
0,Marshall,Marshall Major III On-Ear Headphones (Black) (...,4.1 out of 5,"2,011 ratings","₹ 4,999.00",7 Days Replacement,"Friday, July 2",In stock.,Major III features custom-tuned 40 mm drivers ...,https://www.amazon.in/gp/slredirect/picassoRed...
1,Marshall,Marshall 4090940 Mode EQ in-Ear Headphones (Bl...,4 out of 5,"1,724 ratings","₹ 3,999.00",7 Days Replacement,"Friday, July 2",In stock.,Marshall\nMode eq sports the iconic marshall l...,https://www.amazon.in/gp/slredirect/picassoRed...
2,BoAt,boAt Bassheads 900 On Ear Wired Headphones(Car...,4.2 out of 5,"42,404 ratings",₹ 799.00,7 Days Replacement,"Friday, July 2",In stock.,Has a PVC cable which is durable and tangle fr...,https://www.amazon.in/Boat-BassHeads-900-Wired...
3,ZEBRONICS,Zebronics Zeb-Thunder Wireless BT Headphone Co...,3.9 out of 5,"21,216 ratings",₹ 749.00,7 Days Replacement,"Friday, July 2",In stock.,Comfortable Design: Beat the workday blues wit...,https://www.amazon.in/Zebronics-Zeb-Thunder-Bl...
4,ZEBRONICS,Zebronics Zeb-Thunder Wireless BT Headphone Co...,3.9 out of 5,"21,216 ratings",₹ 699.00,7 Days Replacement,"Friday, July 2",In stock.,Comfortable Design: Beat the workday blues wit...,https://www.amazon.in/Zebronics-Zeb-Thunder-Bl...


In [62]:
#Saving the output
df.to_csv('headphones.csv')

- Q3. Write a python program to access the search bar and search button on images.google.com and scrape 100 images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’.

In [2]:
# Activating the chrome browser
driver=webdriver.Chrome("chromedriver.exe") 
time.sleep(3)

# Opening the google images
url = "https://images.google.com/"
driver.get(url)


search_bar = driver.find_element_by_xpath('//*[@id="sbtc"]/div/div[2]/input')    # Finding the search bar using it's xpath
search_bar.send_keys("fruits")       # Inputing "banana" keyword to search rock images
search_button = driver.find_element_by_xpath('//*[@id="sbtc"]/button')    # Finding the xpath of search button
search_button.click()        # Clicking the search button


# 500 time we scroll down by 10000 in order to generate more images on the website
for _ in range(500):
    driver.execute_script("window.scrollBy(0,10000)")
    
images = driver.find_elements_by_xpath('//img[@class="rg_i Q4LuWd"]')

img_urls = []
img_data = []
for image in images:
    source= image.get_attribute('src')
    if source is not None:
        if(source[0:4] == 'http'):
            img_urls.append(source)
            
            
for i in range(len(img_urls)):
    if i >= 100:
        break
    print("Downloading {0} of {1} images" .format(i, 100))
    response= requests.get(img_urls[i])
    file = open(r"C:/Users/ash.ASHISH/Fliprobo/image folder"+str(i)+".jpg", "wb")
    file.write(response.content)

- 4. Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on www.flipkart.com and scrape following details for all the search results displayed on 1st page. Details to be scraped: “Brand Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”, “Secondary Camera”, “Display Size”, “Display Resolution”, “Processor”, “Processor Cores”, “Battery Capacity”, “Price”, “Product URL”. In case if any of the details is missing then replace it by “- “. Save your results in a dataframe and CSV.

In [12]:
driver=webdriver.Chrome("chromedriver.exe")

In [13]:
url = "https://www.flipkart.com/"
driver.get(url)
# Button to close login popup
try:
    login_button = driver.find_element_by_xpath('//button[@class="_2KpZ6l _2doB4z"]')     
    login_button.click()
except NoSuchElementException : 
    print("No Login page")

In [20]:
# entering “smartphones” in “Search” field 
search_bar=driver.find_element_by_class_name("_3704LK")
search_bar.send_keys('Note 9 ')

time.sleep(2)
#locating the button and clicking it toh search for Smartphone 
button=driver.find_element_by_class_name('L0Z3Pu')
button.click() 

In [21]:
phone_name=[]
price=[]
phone_colour=[]
phone_ram=[]
phone_storage=[]
camera_p=[]
camera_s=[]
display_size=[]
display_resolution=[]
phone_processor=[]
processor_cores=[]
battery_capacity=[]
urls=[]
product_url=[]

# Collecting URLs for each product on the page

for links in driver.find_elements_by_xpath("//a[@class='_1fQZEK']"):
    urls.append(links.get_attribute('href'))
    
purl=driver.find_elements_by_xpath("//a[@class='_1fQZEK']")   
for i in purl:
    product_url.append(i.get_attribute('href'))
    
# Iterating through the URL list and accessing each link to extract data    
for url in urls:
    driver.get(url)
    time.sleep(2)

    # Finding the Read More button and clicking it
    driver.find_element_by_xpath("//button[@class='_2KpZ6l _1FH0tX']").click()
    time.sleep(1)
    
    # Storing all the table rows
    model=driver.find_elements_by_xpath("//tr[@class='_1s_Smc row']")
    
    
    
    try:       
        Price=driver.find_element_by_xpath("//div[@class='_30jeq3 _16Jk6d']")#locating the price
        price.append(Price.text)   
    except NoSuchElementException:
        price.append("--")

    # Iterating through the Table rows to get required data
    for name in model:
        text1=name.find_element_by_xpath("td[1]").text 
    
        #Phone_Processor
        if text1=='Processor Type':
            desc_td = name.find_element_by_xpath("td[2]")

            desc_dtl = desc_td.text if desc_td is not None else "-"

            phone_processor.append(desc_dtl)
            
        # Extracting Model name
        if text1=='Model Name':
            desc_td11 = name.find_element_by_xpath("td[2]")
            desc_phl = desc_td11.text if desc_td11 is not None else "-"
            phone_name.append(desc_phl)

        # Extracting Phone Color
        if text1=='Color':
            desc_td12 = name.find_element_by_xpath("td[2]")
            desc_cl2 = desc_td12.text if desc_td12 is not None else "-"
            phone_colour.append(desc_cl2)
            

        # Extracting Display Size
        if text1=='Display Size':
            desc_td13 = name.find_element_by_xpath("td[2]")
            desc_ds = desc_td13.text if desc_td13 is not None else "-"
            display_size.append(desc_ds)

        # Extracting Display Resolution
        if text1=='Resolution':
            try:       
                resolution=name.find_element_by_xpath("td[2]")#locating the product
                display_resolution.append(resolution.text)#appending the ratings in Ratings list   
            except NoSuchElementException:
                display_resolution.append("--")
        
         # Extracting Cores in Processor
        if text1=='Processor Core':
            processor_C = name.find_element_by_xpath("td[2]")

            process_dtl = processor_C.text if processor_C is not None else "-"

            processor_cores.append(process_dtl)
            

        # Extracting Internal Storage(ROM)
        if text1=='Internal Storage':
            desc_td2 = name.find_element_by_xpath("td[2]")
            ph_store1 = desc_td2.text if desc_td2 is not None else "-"
            phone_storage.append(ph_store1)


        # Extracting RAM Memory
        if text1=='RAM':
            desc_td16 = name.find_element_by_xpath("td[2]")
            ph_store2 = desc_td16.text if desc_td16 is not None else "-"
            phone_ram.append(ph_store2)
    

        # Extracting Primary Camera Details
        if text1=='Primary Camera':
            desc_td3 = name.find_element_by_xpath("td[2]")
            ph_store3 = desc_td3.text if desc_td3 is not None else "-"
            camera_p.append(ph_store3)
            

        # Extracting Secondary Camera Details

        if text1=='Secondary Camera':
            desc_td4 = name.find_element_by_xpath("td[2]")
            ph_store4 = desc_td4.text if desc_td4 is not None else "-"
            camera_s.append(ph_store4)
            

        # Extracting Battery Capacity
        if text1=='Battery Capacity':
            desc_td5 = name.find_element_by_xpath("td[2]")
            ph_store5 = desc_td5.text if desc_td5 is not None else "-"
            battery_capacity.append(ph_store5)    
       
            

In [125]:
df=pd.DataFrame()
df['Phone Name']=phone_name
df['Price']=price
df['Colour']=phone_colour
df['RAM']=phone_ram
df['ROM']=phone_storage
df['Primary Camera']=camera_p
df['Secondary Camera']=camera_s
df['Display Size']=display_size
df['Resolution']=display_resolution
df['Processor']=phone_processor
df['Processor core']=processor_cores
df['Battery']=battery_capacity
df['Product Url']=product_url

In [126]:
df

,Phone Name,Price,Colour,RAM,ROM,Primary Camera,Secondary Camera,Display Size,Resolution,Processor,Processor core,Battery,Product Url
0,Note 9,"₹10,999",Pebble Grey,4 GB,64 GB,48MP + 8MP + 2MP + 2MP,13MP Front Camera,16.59 cm (6.53 inch),2340 x 1080 Pixels,MediaTek Helio G85,Octa Core,5020 mAh,https://www.flipkart.com/redmi-note-9-pebble-g...
1,Note 9,"₹10,999",Shadow Black,4 GB,64 GB,48MP + 8MP + 2MP + 2MP,13MP Front Camera,16.59 cm (6.53 inch),2340 x 1080 Pixels,MediaTek Helio G85,Octa Core,5020 mAh,https://www.flipkart.com/redmi-note-9-shadow-b...
2,Note 9,"₹12,442",Arctic White,4 GB,64 GB,48MP + 8MP + 2MP + 2MP,13MP Front Camera,16.59 cm (6.53 inch),2340 x 1080 Pixels,MediaTek Helio G85,Octa Core,5020 mAh,https://www.flipkart.com/redmi-note-9-arctic-w...
3,Note 9,"₹12,444",Aqua Green,4 GB,64 GB,48MP + 8MP + 2MP + 2MP,13MP Front Camera,16.59 cm (6.53 inch),2340 x 1080 Pixels,MediaTek Helio G85,Octa Core,5020 mAh,https://www.flipkart.com/redmi-note-9-aqua-gre...
4,Note 9,"₹16,440",Arctic White,6 GB,128 GB,48MP + 8MP + 2MP + 2MP,13MP Front Camera,16.59 cm (6.53 inch),2340 x 1080 Pixels,MediaTek Helio G85,Octa Core,5020 mAh,https://www.flipkart.com/redmi-note-9-arctic-w...
5,Note 9,"₹16,433",Shadow Black,6 GB,128 GB,48MP + 8MP + 2MP + 2MP,13MP Front Camera,16.59 cm (6.53 inch),2340 x 1080 Pixels,MediaTek Helio G85,Octa Core,5020 mAh,https://www.flipkart.com/redmi-note-9-shadow-b...
6,Note 9,"₹14,799",Arctic White,4 GB,128 GB,48MP + 8MP + 2MP + 2MP,13MP Front Camera,16.59 cm (6.53 inch),2340 x 1080 Pixels,MediaTek Helio G85,Octa Core,5020 mAh,https://www.flipkart.com/redmi-note-9-arctic-w...
7,Note 9,"₹14,988",Pebble Grey,4 GB,128 GB,48MP + 8MP + 2MP + 2MP,13MP Front Camera,16.59 cm (6.53 inch),2340 x 1080 Pixels,MediaTek Helio G85,Octa Core,5020 mAh,https://www.flipkart.com/redmi-note-9-pebble-g...
8,Note 9,"₹13,890",Aqua Green,4 GB,128 GB,48MP + 8MP + 2MP + 2MP,13MP Front Camera,16.59 cm (6.53 inch),2340 x 1080 Pixels,MediaTek Helio G85,Octa Core,5020 mAh,https://www.flipkart.com/redmi-note-9-aqua-gre...
9,Note 9,"₹13,990",Shadow Black,4 GB,128 GB,48MP + 8MP + 2MP + 2MP,13MP Front Camera,16.59 cm (6.53 inch),2340 x 1080 Pixels,MediaTek Helio G85,Octa Core,5020 mAh,https://www.flipkart.com/redmi-note-9-shadow-b...


- Q5 - Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google maps.

In [89]:
driver=webdriver.Chrome("chromedriver.exe")

In [90]:
url = "https://www.google.com/maps/"
driver.get(url)

In [93]:
wait = WebDriverWait(driver, 8)
driver.get(url)
wait.until(EC.element_to_be_clickable((By.ID, "searchboxinput"))).send_keys("New York")
wait.until(EC.element_to_be_clickable((By.ID, "searchbox-searchbutton"))).click()
time.sleep(3)

In [94]:
ActionChains(driver).move_to_element(driver.find_element_by_xpath("//html/body")).context_click().perform()
print(wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, "ul[role='menu']>li div div[class*='text']:nth-of-type(1)"))).text)

40.69715, -74.25987


- Q6. Write a program to scrap details of all the funding deals for second quarter (i.e. July 20 – September 20) from trak.in.

In [22]:
wait = WebDriverWait(driver, 10)
element = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="menu-item-51510"]')))
element.click()# Activating the chrome browser
driver=webdriver.Chrome("chromedriver.exe") 
time.sleep(2)

# opening trak.in
url = "https://trak.in/"
driver.get(url)
time.sleep(2)

# do click on funding deals
button = driver.find_element_by_xpath('//li[@id="menu-item-51510"]/a').get_attribute('href')
driver.get(button)

# Empty lists
fund_dict = {}
fund_dict['Date'] = []
fund_dict['Startup Name'] = []
fund_dict['Industry/Vertical'] = []
fund_dict['Sub-Vertical'] = []
fund_dict['Location'] = []
fund_dict['Investor'] = []
fund_dict['Investment Type'] = []
fund_dict['Amount(in USD)'] = []


for i in range(48,51):
    # Date
    dt = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[2]'.format(i))
    for d in dt:
        fund_dict['Date'].append(d.text)

    # Startup Name
    sn = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[3]'.format(i))
    for n in sn:
        fund_dict['Startup Name'].append(n.text)
    
    # Industry/Vertical
    ind = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[4]'.format(i))
    for n in ind:
        fund_dict['Industry/Vertical'].append(n.text)
    
    # Sub-Vertical
    sv = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[5]'.format(i))
    for s in sv:
        fund_dict['Sub-Vertical'].append(s.text)

    # Location
    loc = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[6]'.format(i))
    for l in loc:
        fund_dict['Location'].append(l.text)
    
    # Investor
    inv = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[7]'.format(i))
    for n in inv:
        fund_dict['Investor'].append(n.text)
    
    # Investment Type
    invt = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[8]'.format(i))
    for n in invt:
        fund_dict['Investment Type'].append(n.text)
    
    # Amount
    amt = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[9]'.format(i))
    for a in amt:
        fund_dict['Amount(in USD)'].append(a.text)
    
fund_df = pd.DataFrame(fund_dict)
fund_df

,Date,Startup Name,Industry/Vertical,Sub-Vertical,Location,Investor,Investment Type,Amount(in USD)
0,15/07/2020,Flipkart,E-commerce,E-commerce,Bangalore,Walmart Inc,M&A,"1,200,000,000"
1,16/07/2020,Vedantu,EduTech,Online Tutoring,Bangalore,Coatue Management,Series D,"100,000,000"
2,16/07/2020,Crio,EduTech,Learning Platform for Developers,Bangalore,021 Capital,pre-Series A,"934,160"
3,14/07/2020,goDutch,FinTech,Group Payments,Mumbai,"Matrix India,Y Combinator, Global Founders Cap...",Seed,"1,700,000"
4,13/07/2020,Mystifly,Airfare Marketplace,"Ticketing, Airline Retailing, and Post-Ticketi...",Singapore and Bangalore,Recruit Co. Ltd.,pre-Series B,"3,300,000"
5,09/07/2020,JetSynthesys,Gaming and Entertainment,Gaming and Entertainment,Pune,Adar Poonawalla and Kris Gopalakrishnan.,Venture-Series Unknown,"400,000"
6,10/07/2020,gigIndia,Marketplace,"Crowd Sourcing, Freelance",Pune,Incubate Fund India and Beyond Next Ventures,pre-Series A,"974,200"
7,15/07/2020,PumPumPum,Automotive Rental,Used Car-leasing platform,Gurgaon,Early Adapters Syndicate,Seed,"292,800"
8,14/07/2020,FLYX,OTT Player,Streaming Social Network,New York and Delhi,"Raj Mishra, founder of AIT Global Inc",pre-Seed,"200,000"
9,13/07/2020,Open Appliances Pvt. Ltd.,Information Technology,Internet-of-Things Security Solutions,Bangalore,Unicorn India Ventures,Venture-Series Unknown,"500,000"


- Q7 - Write a program to scrap all the available details of best gaming laptops from digit.in.

In [37]:
# Activating the chrome browser
driver=webdriver.Chrome("chromedriver.exe") 
time.sleep(2)

# opening www.digit.in
url = "https://www.digit.in/"
driver.get(url)
time.sleep(2)

#clicking on top 10 option 
top_10=driver.find_element_by_xpath("/html/body/div[1]/div[2]/div[4]/ul/li[4]/a")
top_10.click()

time.sleep(2)
#clicking on laptops option
laptops=driver.find_element_by_xpath("/html/body/div[3]/div/div/div[2]/div[5]/div[1]/div/button[2]")
laptops.click()

time.sleep(2)
#best gaming laptops link
best_gaming=driver.find_element_by_xpath("//div[@id='laptops']//div[3]//a")
driver.get(best_gaming.get_attribute('href'))

#intialising lists
name = []
Price = []
OS = []
display = []
processor = []
HDD = []
RAM = []
weight = []
dimension = []
GPU = []

time.sleep(1)
#scraping names
names=driver.find_elements_by_xpath("//div[@class='right-container']/div/a/h3")
for i in names:
    name.append(i.text)

time.sleep(1)
#scraping operating system
os=driver.find_elements_by_xpath("//div[@class='product-detail']/div/ul/li[1]/div/div")
for i in os:
    OS.append(i.text)

time.sleep(1)
#scraping display
displays=driver.find_elements_by_xpath("//div[@class='product-detail']/div/ul/li[2]/div/div")
for i in displays:
    display.append(i.text)

time.sleep(1)
#scraping processor
processors=driver.find_elements_by_xpath("//div[@class='product-detail']/div/ul/li[3]/div/div")
for i in processors:
    processor.append(i.text)

time.sleep(1)
# scraping memory
memories=driver.find_elements_by_xpath("//div[@class='Spcs-details'][1]/table/tbody/tr[6]/td[3]")# extrat HDD and RAM form xpath
for i in memories:
    HDD.append(i.text.split("/")[0])
    RAM.append(i.text.split("/")[1])

time.sleep(1)
# scraping weight
weights=driver.find_elements_by_xpath("//div[@class='Spcs-details'][1]/table/tbody/tr[7]/td[3]")# extrat weight form xpath
for i in weights:
    weight.append(i.text)

time.sleep(1)
# scraping dimension
dimension=[]
dimensions=driver.find_elements_by_xpath("//div[@class='Spcs-details'][1]/table/tbody/tr[8]/td[3]") 
for i in dimensions:
    dimension.append(i.text)

time.sleep(1)
# scraping graphical processor
GPUs=driver.find_elements_by_xpath("//div[@class='Spcs-details'][1]/table/tbody/tr[9]/td[3]") 
for i in GPUs:
    GPU.append(i.text)

time.sleep(1)

#scraping price
price=driver.find_elements_by_xpath("//table[@id='summtable']//tr//td[3]")
for i in price:
    Price.append(i.text)

#Make data frame
df=pd.DataFrame({"Name":name,
                "price":Price,
                "OS":OS,
                "Display":display,
                "HDD":HDD,
                 "RAM":RAM,
                "processor":processor,
                "weight":weight,
                "Dimension":dimension,
                "Graphical processor":GPU})
df

,Name,price,OS,Display,HDD,RAM,processor,weight,Dimension,Graphical processor
0,ALIENWARE AREA 51M R2,N/A,WINDOWS 10 HOME,"17.3"" (1920 X 1080)",1 TB SSD,16 GBGB DDR4,10TH GENERATION INTEL® CORE™ I7-10700 | 2.90 GHZ,4.1,27.65 x 402.6 x 319.14,Intel® UHD Graphics 630
1,ALIENWARE M15 R3,₹341990,WINDOWS 10 HOME,"15.6"" (3840 X 2160)",1 TB SSD,16 GBGB DDR4,10TH GENERATION INTEL® CORE™ I9-10980HK | NA,NA,NA,NA
2,ASUS ROG STRIX SCAR 15,N/A,WINDOWS 10 HOME,"15.6"" (1920 X 1080)",1 TB SSD,16 GBGB DDR4,AMD RYZEN™ 9 5900HX | 3.3 GHZ,2.30,35.4 x 25.9 x 2.26,NVIDIA® GeForce RTX™ 3070
3,ASUS ROG ZEPHYRUS G14,₹164990,WINDOWS 10 HOME,"14"" (1920 X 1080)",1 TB SSD,16 GBGB DDR4,AMD 3RD GENERATION RYZEN 9 | 3.3 GHZ,1.65,32.5 x 22.1 x 1.8,NVIDIA GeForce RTX 2060
4,LENOVO LEGION 5I,₹71990,WINDOWS 10 PRO,"15.6"" (1920 X 1080)",1 TB SSD,16 GBGB DDR4,10TH GENERATION INTEL® CORE™ I5-10300H | 2.50 GHZ,2.3,363.06 x 259.61 x 23.57,NVIDIA® GeForce® GTX 1650 4GB
5,ASUS ROG ZEPHYRUS DUO 15,₹185000,WINDOWS 10,"15.6"" (3840 X 1100)",512 GB SSD,4 GBGB DDR4,INTEL CORE I7 10TH GEN 10875H | NA,2.4,268.30 x 360.00 x 20.90,NVIDIA GeForce RTX 2070 Max-Q
6,ACER ASPIRE 7 GAMING,₹56990,WINDOWS 10 HOME,"15.6"" (1920 X 1080)",512 GB SSD,8 GBGB DDR4,AMD RYZEN™ 5-5500U HEXA-CORE | NA,2.15,2.29 x 36.3 x 25.4,NVIDIA® GeForce® GTX 1650


In [24]:
df.to_csv("gaming laptop list.csv")

* Q8- Write a python program to scrape the details for all billionaires from www.forbes.com. Details to be scrapped: “Rank”, “Name”, “Net worth”, “Age”, “Citizenship”, “Source”, “Industry”.

In [38]:
# Activating the chrome browser
driver=webdriver.Chrome("chromedriver.exe") 
time.sleep(2)

# Opening the forbes.com
url = "https://www.forbes.com/?sh=69e6b8c92254"
driver.get(url)
time.sleep(2)

#clicking the explore button
button = driver.find_element_by_xpath("//button[@class='icon--hamburger']")
button.click()
time.sleep(1)

#select billioners 
billioners = driver.find_element_by_xpath("/html/body/div[1]/header/nav/div[3]/ul/li[1]")
billioners.click()
time.sleep(1)

#select world billioners 
world_billioners= driver.find_element_by_xpath("/html/body/div[1]/header/nav/div[3]/ul/li[1]/div[2]/ul/li[2]/a")
world_billioners.click()
time.sleep(1)


#make empty listes
Rank = [] 
Name = [] 
total_net_worth = [] 
Age = []
country_of_citizenship = [] 
Source = []
industry = []


while(True):
    #scraping rank of billionaire
    rank= driver.find_elements_by_xpath("//div[@class='rank']")
    for i in rank:
        Rank.append(i.text)
    time.sleep(1)
    
    #scraping name of billionaire
    name= driver.find_elements_by_xpath("//div[@class='personName']//div")
    for i in name:
        Name.append(i.text)
    time.sleep(1)
    
     #scraping Age of billionaire
    age= driver.find_elements_by_xpath("//div[@class='age']//div")
    for i in age:
        Age.append(i.text)   
    time.sleep(1)
    
    #scraping citizenship of billionaire    
    citizenship= driver.find_elements_by_xpath("//div[@class='countryOfCitizenship']")
    for i in citizenship:
        country_of_citizenship.append(i.text)
    time.sleep(1)
    
    #scraping source of income
    source= driver.find_elements_by_xpath("//div[@class='source']")
    for i in source:
        Source.append(i.text)    
    time.sleep(1)
        
    #scraping Age of billionaire
    industries= driver.find_elements_by_xpath("//div[@class='category']//div")
    for i in industries:
        industry.append(i.text)
        
    #scraping net_worth of billionaire
    net_worth= driver.find_elements_by_xpath("//div[@class='netWorth']//div[1]")
    for i in net_worth:
        total_net_worth.append(i.text)
    time.sleep(1)    
        
    try:
        next_button = driver.find_element_by_xpath("//button[@class='pagination-btn pagination-btn--next ']")
        next_button.click()
    except:
        break  

        
worth = []        
for i in range(0,len(total_net_worth),2):
        worth.append(total_net_worth[i])
    

time.sleep(2)        


In [39]:
#creating dataframe
df=pd.DataFrame({'Rank':Rank,
                'Name': Name,
                'Net Worth':Net_Worth,
                'Age':Age,
                'Country of Citizenship':country_of_citizenship,
                'Source':Source,
                'Industry':industry})
df

,Rank,Name,Net Worth,Age,Country of Citizenship,Source,Industry
0,1.,Jeff Bezos,$177 B,57,United States,Amazon,Technology
1,2.,Elon Musk,$151 B,49,United States,"Tesla, SpaceX",Automotive
2,3.,Bernard Arnault & family,$150 B,72,France,LVMH,Fashion & Retail
3,4.,Bill Gates,$124 B,65,United States,Microsoft,Technology
4,5.,Mark Zuckerberg,$97 B,36,United States,Facebook,Technology
...,...,...,...,...,...,...,...
2750,2674.,Daniel Yong Zhang,$1 B,49,China,e-commerce,Technology
2751,2674.,Zhang Yuqiang,$1 B,65,China,Fiberglass,Manufacturing
2752,2674.,Zhao Meiguang,$1 B,58,China,gold mining,Metals & Mining
2753,2674.,Zhong Naixiong,$1 B,58,China,conglomerate,Diversified


- Q9. Write a program to extract at least 500 Comments, Comment upvote and time when comment was posted from any YouTube Video.

In [41]:
# Activating the chrome browser
driver=webdriver.Chrome("chromedriver.exe")
time.sleep(2)

# Opening the youtube.com
url = "https://www.youtube.com/"
driver.get(url)
time.sleep(2)

#finding element for search bar
search_bar = driver.find_element_by_id('search')
search_bar.send_keys("Despacito")  #entering Video name
time.sleep(1)

#clicking on search button
search_btn = driver.find_element_by_id("search-icon-legacy")  
search_btn.click()
time.sleep(1)

#clicking on first video
link_click = driver.find_element_by_xpath("//yt-formatted-string[@class ='style-scope ytd-video-renderer']")
link_click.click()

In [42]:
# 1000 time we scroll down by 10000 in order to generate more Comments
for i in range(1000):
    driver.execute_script("window.scrollBy(0,10000)")

In [44]:
#make empty lists
comments = []
comment_time = []
Time = []
Likes = []
No_of_Likes = []

#scrape comments
cm = driver.find_elements_by_id("content-text")
for i in cm:
    if i.text is None:
        comments.append("--")
    else:
        comments.append(i.text)
time.sleep(5)


# scrape time when comment was posted
tm = driver.find_elements_by_xpath("//a[contains(text(),'ago')]")
for i in tm:
    Time.append(i.text)

for i in range(0,len(Time),2):
    comment_time.append(Time[i])
time.sleep(5)
    
# scrape the comment likes
like = driver.find_elements_by_xpath("//span[@class='style-scope ytd-comment-action-buttons-renderer']")
for i in like:
    Likes.append(i.text)
    
for i in range(1,len(Likes),2):
    No_of_Likes.append(Likes[i])
    
    
time.sleep(2)
#creating dataframe
df=pd.DataFrame({'Comments':comments,
                 'Time':comment_time,
                'Likes':No_of_Likes})
df

,Comments,Time,Likes
0,2017: People came to listen song. \n2021: Peop...,3 months ago,167K
1,I am wondering how many people would come to w...,2 days ago,1.9K
2,fun fact: this vid is so popular that the WHOL...,15 hours ago,115
3,so basically everyone on the world should know...,1 day ago,359
4,I couldn't understand a single word but still ...,1 day ago,220
5,2018 : T-series VS pewdiepie\n2019 : Avengers ...,10 months ago (edited),19K
6,The person who wrote the first comment on this...,1 day ago,179
7,No matter how many hit songs came but this one...,14 hours ago,36
8,This song hits different with those flashbacks!!!,1 day ago,196
9,Ay \nFonsi \nDY\nOh-oh,1 day ago (edited),109


* Q 10. Write a python program to scrape a data for all available Hostels from https://www.hostelworld.com/ in “London” location. You have to scrape hostel name, distance from city centre, ratings, total reviews, overall reviews, privates from price, dorms from price, facilities and property description

In [42]:
# Activating the chrome browser
driver=webdriver.Chrome("chromedriver.exe") 
time.sleep(3)

# Opening the homepage of hostelworld.com
url = "https://www.hostelworld.com/"
driver.get(url)

In [43]:
#locating the location search bar
search_loc = driver.find_element_by_xpath('//*[@id="search-input-field"]')
search_loc.send_keys("London")
time.sleep(2)

In [44]:
place = driver.find_element_by_xpath('//*[@id="predicted-search-results"]/li[2]')
place.click()
time.sleep(2)

#click in search button
search_btn = driver.find_element_by_id('search-button')
search_btn.click()

In [45]:
# Make empty lists
Hostel_Name = []
Distance = []
overall_review = []
total_reviews = []
facilities = []
price = []
Rating = []
property_description = []

while(True):
    
    # Hostel name
    names = driver.find_elements_by_xpath('//h2[@class="title title-6"]')
    for name in names:
        Hostel_Name.append(name.text)
    time.sleep(2)
        
    # Distance from city
    distance = driver.find_elements_by_xpath('//span[@class="description"]')
    for d in distance:
        Distance.append(d.text)
    time.sleep(2)
        
    #Review    
    review = driver.find_elements_by_xpath('//div[@class="keyword"]//span')
    for r in review:
        overall_review.append(r.text)
    time.sleep(2)
    
    # Total number of reviews     
    t_review = driver.find_elements_by_xpath('//div[@class="reviews"]')
    for t in t_review:
        total_reviews.append(t.text)
    time.sleep(2)
    
    # facilities
    service = driver.find_elements_by_xpath('//div[@class="facilities-label facilities"]')
    for s in service:
        facilities.append(s.text)
    time.sleep(2)
    
    # Prices    
    prices = driver.find_elements_by_xpath('//div[@class="price-col"]')
    for p in prices:
        price.append(p.text)
    time.sleep(2)    
        
    try:
        next_button = driver.find_element_by_xpath('//div[@class="pagination-item pagination-next"]')
        next_button.click()
    except:
        break
        
time.sleep(2) 

# Separate  Privates_From price  and  Dorms_From price
private = []
for i in range(0,len(price),2):
    private.append(price[i])
time.sleep(2)

dorms = []
for i in range(1,len(price),2):
    dorms.append(price[i]) 

In [46]:
# scrape Hostels URL
hostel_url = []

while(True):
    urls = driver.find_elements_by_xpath('//h2[@class="title title-6"]/a')
    for url in urls:
        hostel_url.append(url.get_attribute("href"))
    time.sleep(2)    
        
    try:
        next_button = driver.find_element_by_xpath('//div[@class="pagination-item pagination-prev"]')
        next_button.click()
    except:
        break
        
Rate = []
for page in hostel_url:
    driver.get(page)
    
    # Rating
    try:
        ratings = driver.find_element_by_xpath('//*[@id="__layout"]/div/div[1]/section/div[6]/div/div[1]/div[1]/div[1]')
        Rate.append(ratings.text)
    except NoSuchElementException:
        Rate.append("No Rating")  
    time.sleep(2)
    
    # Property Description
    try:
        pd = driver.find_element_by_xpath('//*[@id="__layout"]/div/div[1]/section/div[6]/div/div[2]/div[2]/div/div[2]')
        property_description.append(pd.text)
    except NoSuchElementException:
        property_description.append("No Description")  

    
time.sleep(2)        
# remove extra data from Rating     
all_text = []
for i in Rate:
    all_text.append(i.split())
time.sleep(2)

for i in all_text:
    Rating.append(i[0]) 

In [50]:
#creating dataframe
df = pd.DataFrame({'Hostel Name':Hostel_Name,
                'Distance from city centre':Distance,
                'Overall Review':overall_review[3:],
                'Total Reviews':total_reviews,
                'Facilities':facilities,
                'Privates From Price':private,
                'Dorms From Price':dorms,
                'Rating':Rating,
                'Property Description':property_description})
df

,Hostel Name,Distance from city centre,Overall Review,Total Reviews,Facilities,Privates From Price,Dorms From Price,Rating,Property Description
0,St Christopher's Village,Hostel - 1.8km from city centre,Fabulous,10819 Total Reviews,Free WiFi\nFollows Covid-19 sanitation guidance,Privates From\nRs5571,Dorms From\nRs1569,9.7,No Description
1,Generator London,Hostel - 3km from city centre,Very Good,6732 Total Reviews,Free WiFi\nFollows Covid-19 sanitation guidance,Privates From\nRs8492,Dorms From\nRs2004,No,No Description
2,Safestay London Kensington Holland Park,Hostel - 5.9km from city centre,Superb,1074 Total Reviews,Free WiFi,No Privates Available,Dorms From\nRs1101,No,No Description
3,"PubLove @ The Crown, Battersea",Hostel - 4.7km from city centre,Good,207 Total Reviews,Free WiFi\nFollows Covid-19 sanitation guidance,No Privates Available,Dorms From\nRs1597,No,No Description
4,247london Hostel and Private Rooms,Hostel - 8.2km from city centre,No Rating,443 Total Reviews,Free WiFi,Privates From\nRs5197,No Dorms Available,No,No Description
...,...,...,...,...,...,...,...,...,...
76,Park Hotel Essex,Hotel - 24.1km from city centre,No Rating,108 Total Reviews,Free Breakfast\nFollows Covid-19 sanitation gu...,Privates From\nRs3839,No Dorms Available,8.9,"Located a short walk from Notting Hill, Hyde P..."
77,Cranbrook Hotel,Hotel - 14.8km from city centre,No Rating,58 Total Reviews,Free Breakfast\nFollows Covid-19 sanitation gu...,Privates From\nRs3057,No Dorms Available,7.2,No Description
78,St. Athans,Bed and Breakfast - 2.9km from city centre,No Rating,234 Total Reviews,Free WiFi\nFollows Covid-19 sanitation guidance,Privates From\nRs3845,No Dorms Available,4.9,No Description
79,Aron Guest House,Bed and Breakfast - 13.1km from city centre,No Rating,26 Total Reviews,Free WiFi,Privates From\nRs6624,No Dorms Available,8.0,No Description
